# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("C:/Users/refle/Data_Bootcamp/Data_analytics_challenges/Starter_Code_API_challenge/output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cidreira,-30.1811,-50.2056,20.54,63,97,2.91,BR,1692195098
1,1,badger,64.8000,-147.5333,14.28,83,75,0.00,US,1692195098
2,2,piti village,13.4626,144.6933,28.49,86,0,2.06,GU,1692195099
3,3,bereeda,11.8704,51.0579,33.09,49,16,0.66,SO,1692195100
4,4,ushuaia,-54.8000,-68.3000,-0.74,80,23,1.19,AR,1692195100


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points(
  "Lng",
  "Lat",
  geo = True,
  tiles = "OSM",
  frame_width = 700,
  frame_height = 500,
  size = "Humidity",
  color = "City",
  scale = 1,
  alpha = 0.5
)

# Display the map
city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 23) &  
    (city_data_df["Max Temp"] < 30) &  
    (city_data_df["Wind Speed"] < 3.0) &  
    (city_data_df["Cloudiness"] == 0) 
]

# Drop any rows with null values
perfect_weather_df = ideal_weather_df.dropna()

# Display sample data
perfect_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,piti village,13.4626,144.6933,28.49,86,0,2.06,GU,1692195099
36,36,hamilton,39.1834,-84.5333,23.19,79,0,2.57,US,1692194995
39,39,laguna,38.4210,-121.4238,27.02,75,0,1.54,US,1692195121
43,43,santa ana de yacuma,-13.7500,-65.4333,29.86,40,0,2.30,BO,1692195124
79,79,guankou,28.1500,113.6333,27.05,75,0,0.48,CN,1692195158
95,95,suozhen,36.9539,118.1047,26.88,58,0,1.26,CN,1692195168
263,263,frontera,26.9333,-101.4500,29.38,52,0,0.89,MX,1692195290
286,286,hanceville,34.0607,-86.7675,25.86,65,0,2.17,US,1692195051
297,297,san borja,-14.8167,-66.8500,29.12,44,0,1.59,BO,1692195313
366,366,dereli,40.7378,38.4493,26.09,70,0,2.64,TR,1692195356


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,piti village,GU,13.4626,144.6933,86,
36,hamilton,US,39.1834,-84.5333,79,
39,laguna,US,38.4210,-121.4238,75,
43,santa ana de yacuma,BO,-13.7500,-65.4333,40,
79,guankou,CN,28.1500,113.6333,75,
95,suozhen,CN,36.9539,118.1047,58,
263,frontera,MX,26.9333,-101.4500,52,
286,hanceville,US,34.0607,-86.7675,65,
297,san borja,BO,-14.8167,-66.8500,44,
366,dereli,TR,40.7378,38.4493,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius},{lng},{lat}"
    params["bias"] =  f"proximity:{lng},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
piti village - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
santa ana de yacuma - nearest hotel: No hotel found
guankou - nearest hotel: No hotel found
suozhen - nearest hotel: No hotel found
frontera - nearest hotel: No hotel found
hanceville - nearest hotel: No hotel found
san borja - nearest hotel: No hotel found
dereli - nearest hotel: No hotel found
pangoa - nearest hotel: No hotel found
callaway - nearest hotel: No hotel found
washington - nearest hotel: No hotel found
cabot - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,piti village,GU,13.4626,144.6933,86,No hotel found
36,hamilton,US,39.1834,-84.5333,79,No hotel found
39,laguna,US,38.4210,-121.4238,75,No hotel found
43,santa ana de yacuma,BO,-13.7500,-65.4333,40,No hotel found
79,guankou,CN,28.1500,113.6333,75,No hotel found
95,suozhen,CN,36.9539,118.1047,58,No hotel found
263,frontera,MX,26.9333,-101.4500,52,No hotel found
286,hanceville,US,34.0607,-86.7675,65,No hotel found
297,san borja,BO,-14.8167,-66.8500,44,No hotel found
366,dereli,TR,40.7378,38.4493,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
Hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

Hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)